In [4]:
import numpy as np
import joblib
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [6]:
# Load all the saved models
random_forest = joblib.load(
    "../../../Models/requirement_relevancy_experiment/classifier_models/distilbert_random_forest_classifier.joblib"
)

gradient_boosting = joblib.load(
    "../../../Models/requirement_relevancy_experiment/classifier_models/distilbert_gradient_boost_classifier.joblib"
)

ada_boost = joblib.load(
    "../../../Models/requirement_relevancy_experiment/classifier_models/distilbert_adaboost_classifier.joblib"
)

xgboost = joblib.load(
    "../../../Models/requirement_relevancy_experiment/classifier_models/distilbert_xgboost_classifier.joblib"
)

In [8]:
# Use this if you already have data from the model state outputs
X_train = np.loadtxt(
    "../../../Datasets/irrelevant_requirements_dataset/model_state_outputs/distilbert/reshaped_X_train_last_hidden_states.csv"
)

X_test = np.loadtxt(
    "../../../Datasets/irrelevant_requirements_dataset/model_state_outputs/distilbert/reshaped_X_test_last_hidden_states.csv"
)

y_train = np.loadtxt(
    "../../../Datasets/irrelevant_requirements_dataset/model_state_outputs/distilbert/y_train.csv"
)

y_test = np.loadtxt(
    "../../../Datasets/irrelevant_requirements_dataset/model_state_outputs/distilbert/y_test.csv"
)

In [ ]:
# Load the resampled data

# X_resampled = np.loadtxt(
#     "../../../Datasets/irrelevant_requirements_dataset/distilbert_X_resampled.csv",
#     delimiter=",",
# )

# y_resampled = np.loadtxt(
#       "../../../Datasets/irrelevant_requirements_dataset/distilbert_y_resampled.csv",
#     delimiter=",",
# )

In [ ]:
# X_resampled.shape, y_resampled.shape

((1114, 49152), (1114,))

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X_resampled, y_resampled, test_size=0.2, random_state=42
# )

In [9]:
validation_fold = 5

In [10]:
cross_validation_result_dict = {
    "model_name": [],
    "accuracy": [],
    "precision": [],
    "recall": [],
    "f1": [],
    "fit_time": [],
    "score_time": [],
}


def append_scores(
    cross_validation_result_dict,
    model_name: str,
    accuracy: float,
    precision: float,
    recall: float,
    f1: float,
    fit_time: float,
    score_time: float,
):
    cross_validation_result_dict["model_name"].append(model_name)
    cross_validation_result_dict["accuracy"].append(accuracy)
    cross_validation_result_dict["precision"].append(precision)
    cross_validation_result_dict["recall"].append(recall)
    cross_validation_result_dict["f1"].append(f1)
    cross_validation_result_dict["fit_time"].append(fit_time)
    cross_validation_result_dict["score_time"].append(score_time)

In [11]:
diagonal_line = pd.DataFrame(
    {"False Positive Rate": [0, 1], "True Positive Rate": [0, 1]}
)


def show_roc_curve(model, model_name: str, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    true_positive_rate, false_positive_rate, _ = roc_curve(y_test, y_pred_proba)
    area_under_curve = roc_auc_score(y_test, y_pred_proba)

    fig = px.line(
        diagonal_line,
        x="False Positive Rate",
        y="True Positive Rate",
        title="ROC Curve",
    )

    fig.update_layout(
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        font=dict(size=16),
        legend=dict(
            bordercolor="Black",
            borderwidth=2,
            font=dict(size=18),
        ),
    )

    fig.update_xaxes(showline=True, linewidth=2, linecolor="black", mirror=True)

    fig.update_yaxes(showline=True, linewidth=2, linecolor="black", mirror=True)

    fig.add_trace(
        go.Scatter(
            x=true_positive_rate,
            y=false_positive_rate,
            mode="lines",
            name=f"{model_name} (AUC = {area_under_curve:.4f})",
        )
    )

    fig.show()

## Cross Validation

Cross validation is a technique for assessing how the results of a statistical analysis will generalize to an independent data set. It is mainly used in settings where the goal is prediction, and one wants to estimate how accurately a predictive model will perform in practice. One round of cross-validation involves partitioning a sample of data into complementary subsets, performing the analysis on one subset (called the training set), and validating the analysis on the other subset (called the validation set or testing set). To reduce variability, multiple rounds of cross-validation are performed using different partitions, and the validation results are averaged over the rounds.

### Cross Validation for Random Forest

In [12]:
random_forest_scores = cross_validate(
    random_forest,
    X_train,
    y_train,
    cv=validation_fold,
    n_jobs=-1,
    scoring=["f1", "accuracy", "precision", "recall"],
)

In [13]:
random_forest_scores

{'fit_time': array([ 1.58110213, 10.88836694, 11.10036731, 10.86637163, 10.83236504]),
 'score_time': array([0.02652264, 0.03000021, 0.02700663, 0.02799487, 0.03200746]),
 'test_f1': array([0.78362573, 1.        , 1.        , 1.        , 1.        ]),
 'test_accuracy': array([0.7238806, 1.       , 1.       , 1.       , 1.       ]),
 'test_precision': array([0.64423077, 1.        , 1.        , 1.        , 1.        ]),
 'test_recall': array([1., 1., 1., 1., 1.])}

In [14]:
append_scores(
    cross_validation_result_dict,
    "Random Forest",
    np.mean(random_forest_scores["test_accuracy"]),
    np.mean(random_forest_scores["test_precision"]),
    np.mean(random_forest_scores["test_recall"]),
    np.mean(random_forest_scores["test_f1"]),
    np.mean(random_forest_scores["fit_time"]),
    np.mean(random_forest_scores["score_time"]),
)

In [15]:
show_roc_curve(random_forest, "Random Forest", X_test, y_test)

### Adaboost Cross Validation

In [16]:
adaboost_cross_validation_scores = cross_validate(
    ada_boost,
    X_train,
    y_train,
    cv=validation_fold,
    n_jobs=-1,
    scoring=["f1", "accuracy", "precision", "recall"],
)

adaboost_cross_validation_scores

{'fit_time': array([  4.171808  , 219.13132524, 213.95018959, 216.16019058,
        216.16418839]),
 'score_time': array([0.02599812, 0.67801738, 0.762007  , 0.87403178, 0.88703609]),
 'test_f1': array([0.77647059, 0.96124031, 1.        , 0.98484848, 0.99236641]),
 'test_accuracy': array([0.71641791, 0.96268657, 1.        , 0.98496241, 0.9924812 ]),
 'test_precision': array([0.6407767, 1.       , 1.       , 1.       , 1.       ]),
 'test_recall': array([0.98507463, 0.92537313, 1.        , 0.97014925, 0.98484848])}

In [17]:
append_scores(
    cross_validation_result_dict,
    "AdaBoost",
    np.mean(adaboost_cross_validation_scores["test_accuracy"]),
    np.mean(adaboost_cross_validation_scores["test_precision"]),
    np.mean(adaboost_cross_validation_scores["test_recall"]),
    np.mean(adaboost_cross_validation_scores["test_f1"]),
    np.mean(adaboost_cross_validation_scores["fit_time"]),
    np.mean(adaboost_cross_validation_scores["score_time"]),
)

In [18]:
show_roc_curve(ada_boost, "AdaBoost", X_test, y_test)

### XGBoost Cross Validation

In [19]:
xgboost_scores = cross_validate(
    xgboost,
    X_train,
    y_train,
    cv=validation_fold,
    scoring=["f1", "accuracy", "precision", "recall"],
)

xgboost_scores

{'fit_time': array([14.86999607, 29.96797132, 31.70900178, 32.78808832, 32.78355837]),
 'score_time': array([0.05499768, 0.05399632, 0.06000066, 0.0539968 , 0.04999709]),
 'test_f1': array([0.78362573, 1.        , 1.        , 1.        , 1.        ]),
 'test_accuracy': array([0.7238806, 1.       , 1.       , 1.       , 1.       ]),
 'test_precision': array([0.64423077, 1.        , 1.        , 1.        , 1.        ]),
 'test_recall': array([1., 1., 1., 1., 1.])}

In [20]:
append_scores(
    cross_validation_result_dict,
    "XGBoost",
    np.mean(xgboost_scores["test_accuracy"]),
    np.mean(xgboost_scores["test_precision"]),
    np.mean(xgboost_scores["test_recall"]),
    np.mean(xgboost_scores["test_f1"]),
    np.mean(xgboost_scores["fit_time"]),
    np.mean(xgboost_scores["score_time"]),
)

In [21]:
show_roc_curve(xgboost, "XGBoost", X_test, y_test)

### Gradient Boosting Cross Validation


In [22]:
gradient_boosting_cross_validation_scores = cross_validate(
    gradient_boosting,
    X_train,
    y_train,
    cv=validation_fold,
    n_jobs=-1,
    scoring=["f1", "accuracy", "precision", "recall"],
)

In [23]:
append_scores(
    cross_validation_result_dict,
    "Gradient Boosting",
    np.mean(gradient_boosting_cross_validation_scores["test_accuracy"]),
    np.mean(gradient_boosting_cross_validation_scores["test_precision"]),
    np.mean(gradient_boosting_cross_validation_scores["test_recall"]),
    np.mean(gradient_boosting_cross_validation_scores["test_f1"]),
    np.mean(gradient_boosting_cross_validation_scores["fit_time"]),
    np.mean(gradient_boosting_cross_validation_scores["score_time"]),
)

In [31]:
show_roc_curve(gradient_boosting, "Gradient Boost", X_test, y_test)

## Result Export

In [25]:
cross_validation_result_dict

{'model_name': ['Random Forest', 'AdaBoost', 'XGBoost', 'Gradient Boosting'],
 'accuracy': [0.9447761194029851,
  0.9313096173268993,
  0.9447761194029851,
  0.9432835820895523],
 'precision': [0.9288461538461539,
  0.9281553398058253,
  0.9288461538461539,
  0.9288461538461539],
 'recall': [1.0, 0.9730890999547717, 1.0, 0.9970149253731343],
 'f1': [0.9567251461988304,
  0.9429851590750078,
  0.9567251461988304,
  0.9552213868003342],
 'fit_time': [9.053714609146118,
  173.91554036140442,
  28.42372317314148,
  1054.9498009681702],
 'score_time': [0.02870635986328125,
  0.6454180717468262,
  0.05459771156311035,
  0.027600526809692383]}

In [26]:
from pandas import DataFrame

cross_validation_result_df = DataFrame(cross_validation_result_dict)
cross_validation_result_df.to_csv(
    "../../../Results/irrelevant_requirements_experiment/classifiers_with_distilbert_cross_validation_results.csv",
    index=False,
)

In [27]:
cross_validation_result_df

,model_name,accuracy,precision,recall,f1,fit_time,score_time
0,Random Forest,0.944776,0.928846,1.000000,0.956725,9.053715,0.028706
1,AdaBoost,0.931310,0.928155,0.973089,0.942985,173.915540,0.645418
2,XGBoost,0.944776,0.928846,1.000000,0.956725,28.423723,0.054598
3,Gradient Boosting,0.943284,0.928846,0.997015,0.955221,1054.949801,0.027601


In [30]:
# Make a bar chart of the cross validation results that contains the accuracy, precision, recall, and f1 scores. The graph should also contain value of each of the scores.

import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Bar(
            name="Accuracy",
            x=cross_validation_result_df["model_name"],
            y=cross_validation_result_df["accuracy"],
        ),
        go.Bar(
            name="Precision",
            x=cross_validation_result_df["model_name"],
            y=cross_validation_result_df["precision"],
        ),
        go.Bar(
            name="Recall",
            x=cross_validation_result_df["model_name"],
            y=cross_validation_result_df["recall"],
        ),
        go.Bar(
            name="F1",
            x=cross_validation_result_df["model_name"],
            y=cross_validation_result_df["f1"],
        ),
    ]
)

fig.update_layout(
    barmode="group",
    xaxis_title="Model",
    yaxis_title="Score",
    font=dict(size=16),
    legend=dict(
        bordercolor="Black",
        borderwidth=2,
        font=dict(size=18),
    ),
)

fig.update_xaxes(showline=True, linewidth=2, linecolor="black", mirror=True)

fig.update_yaxes(showline=True, linewidth=2, linecolor="black", mirror=True)

fig.show()